In [ ]:
#!pip install transformers
#!pip install --upgrade unsloth unsloth_zoo

In [19]:

!wget --no-check-certificate "https://jpdata1-my.sharepoint.com/:u:/g/personal/anand_jpdata_co/EVQs7sQlMxRHvHBMhonrXuIBSdosZ5CQPn2S6WG4I1uuhg?e=Ncv8WW&download=1" -O MAAI_gpt2_extra_large_torch_50

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--2025-10-09 13:55:29--  https://jpdata1-my.sharepoint.com/:u:/g/personal/anand_jpdata_co/EVQs7sQlMxRHvHBMhonrXuIBSdosZ5CQPn2S6WG4I1uuhg?e=Ncv8WW&download=1
Resolving jpdata1-my.sharepoint.com (jpdata1-my.sharepoint.com)... 13.107.136.10, 13.107.138.10, 2620:1ec:8f8::10, ...
Connecting to jpdata1-my.sharepoint.com (jpdata1-my.sharepoint.com)|13.107.136.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/anand_jpdata_co/Documents/gpt-weights/MAAI_gpt2_extra_large_torch-50?ga=1 [following]
--2025-10-09 13:55:29--  https://jpdata1-my.sharepoint.com/personal/anand_jpdata_co/Documents/gpt-weights/MAAI_gpt2_extra_large_torch-50?ga=1
Reusing existing connection to jpdata1-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 6281147170 (5.8G) [application/octet-stream]
Saving to: ‘MAAI_gpt2_extra_large_torch_50’

MAAI_gpt2_extra_lar 100%[===================>]   5.85G  70.7MB/s    in 81s     

2025-10-09 13:56:50 (74.3 MB/s) - ‘MA

In [20]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM

import time
import os

#os.environ["PYTORCH_CUDA_ALLOC_CONF = expandable_segments:True"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

#!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'
# --- 1. Configuration ---
# You can adjust these parameters
MODEL_NAME = "gpt2-xl"
MAX_LENGTH = 1024 # Maximum sequence length for GPT-2 Medium
BATCH_SIZE = 4
LEARNING_RATE = 5e-5
EPOCHS = 1
NUM_WORKERS = 4 # For faster data loading
PATH= 'MAAI_gpt2_extra_large_torch_50'

# --- 2. Device Setup (Essential for PyTorch) ---
# Automatically detects and uses GPU (CUDA/ROCm) or falls back to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"PyTorch using device: {device}")


PyTorch using device: cuda


In [21]:

# --- 5. Initialization ---

# Load the Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# Set the padding token, which is necessary for batching
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load the Model and move it to the device (GPU or CPU)
#model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

# Use the lines below for loading stored weights and comment the line above
model = torch.load(PATH, weights_only=False)



In [22]:
prompt = "तुमच्या"
max_new_tokens = 1024

# 1. Tokenize input and move to device
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)



In [23]:
# 2. Generate text
print(f"Generating continuation for prompt: '{prompt}'")
        # Use model.generate() for text generation
with torch.no_grad():
    output_sequences = model.generate(
    input_ids=input_ids,
    max_length=max_new_tokens, # Set max total length
    num_return_sequences=1,
    compile_config=None,
    do_sample=True, # Use sampling for more creative output
    top_k=50,
    top_p=0.95,
    temperature=0.9,
    pad_token_id=tokenizer.eos_token_id # Ensure generation stops correctly
    )

# 3. Decode and Print Output
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print("-" * 20)
print(f"Generated Text:\n{generated_text}")
print("-" * 20)

Generating continuation for prompt: 'तुमच्या'
--------------------
Generated Text:
तुमच्या ताकदीवर अनेक शब्द घेत असेल तर याचे पलन करता येते. मी इतका आहे की त्याचा विचार करून सर्व पक्षातील सोबत येतो. येतो तेव्हा लोकं संकटलेले असताना ते माझ्यासमोर फार कमी बोलतात. त्यामुळे लोकांचे नुकसान होत असते. आमच्यापासून ते शब्द चालत आहे. माझ्यामुळे मी येतो आणि तेथे येतो निर्णय घेतला असता तर अनेक नेत्यांनी स्वतःच्या बाजूला भाव निर्माण केला. कारण असता तर पाच वर्षातील मुलीच्या घरात फार आहे. आम्ही काम करता आणि त्याने पालकमंत्र्यांना स्थगिती दिली. ज्याची निष्ठा होती त्याची विचारधारा गाडीखाली आहे. विचारधारा आणि वेगवेगळ्या पक्षातून हे दोन गट पडत असतात अशी भावना अनेक मंत्र्यांची झाली. मात्र अध्यक्षांच्या हातात ही माहिती जमा पडते. त्यानुसार प्रकाश आंबेडकर या
--------------------
